### IceBreaker is a mobile application that uses the power of GPT-3 to facilitate meaningful connections at networking events. The app generates relevant and engaging questions tailored to the user's interests and event context, making it easy for users to break the ice, have deeper conversations, and build lasting relationships with other attendees.

In [1]:
import os 
import os.path
import pandas as pd
from dotenv import load_dotenv
import openai
import json
from backoff import on_exception, expo
import re
import ast
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")


openai.api_key = os.getenv("OPENAI_API_KEY")
load_dotenv()

COMPLETIONS_MODEL = "gpt-4"

COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "model": COMPLETIONS_MODEL,
}

def count_tokens(text: str) -> int:
    """count the number of tokens in a string"""
    return len(tokenizer.encode(text))

@on_exception(expo, openai.error.InvalidRequestError, max_tries=3)
def get_openai_response(messages):
    response = openai.ChatCompletion.create(messages=messages, **COMPLETIONS_API_PARAMS)
    return response

In [2]:
general_system_message = [
    {
        "role": "system",
        "content": """You are NetworkGenius, an innovative networking AI model. Your job is to help facilitate relevant, meaningful and personal connections at
                events by generating personalized and engaging questions for users, making it easy to break the ice and have deep conversations about careers, topics, relationships, and personal well-being. 
                You are able to analyze user profiles, event context, and user feedback to continuously improve your question-generation capabilities.""",
    },
    {
        "role": "assistant",
        "content": "Please tell me a bit about yourself, such as your industry, job title, interests, and goals.",
    },
]

### Event related information:

In [3]:
event_description = """ This symposium will focus on the role of nurses in demystifying and embracing Artificial Intelligence (AI). Nurses play
        a pivotal role in identifying innovative, applicable, and equitable technology in healthcare. How do informatics nurses forge an innovative future through AI?
Targeted Audience: Nurses, Healthcare professionals and leaders, students, Health IT professionals, Innovators.
Topics and Speakers:
1. Artificial Intelligence (AI) in Healthcare Fireside Chat: Tom Lawry and Kathleen McGrow;
2. Applications of AI in nursing practice: Interdisciplinary Panel
Dont miss this opportunity to learn and network while enjoying hors doeuvres and drinks!!
2. 0 Contact Hours: IntelyCare is accredited as a provider of nursing continuing professional development by the American Nurses Credentialing Centers Commission on Accreditation. © 2023 IntelyCare, Inc. All Rights Reserved.
Interested in more on AI and Healthcare? Watch your confirmation email for discounts on these books:
Hacking Healthcare, How AI and the Intelligence Revolution Will Reboot an Ailing System, By Tom Lawry
AI in Health, A Leaders Guide to Winning in the New Age of Intelligent Health Systems, By Tom Lawry
The Rebel Nurse Handbook, Inspirational Stories by Shift Disruptors, SONSIEL"""

speaker_1 =  {
    "role": "user",
    "content": """ Meghan Reading Turchioe is an Assistant Professor at Columbia University School of Nursing who 
                studies the use of consumer health technology to support patient and clinician decision-making around chronix condition management. 
                She applies machine learning methos to patient-generated health data and electronic health records to generate data-driven insights for
                oatients and clinicians. She holds a phD from Columbia University School of Nursing, MPH from George Washington University, and BSN from
                Boston College and completed her postdoc training in informatics at Weill Cornell Medicine."""
}

speaker_2 = {
    "role": "user",
    "content": """ Robbie Freeman is the Informatics Officer (CNIO) at the Mount Sinai Health System, where he drives the technology stategy to support
                nursing practive and ptient care across hospital and ambulatory care settings. He oversees the clinical data science and digital engagement teams and leads the nursing informatics
                program. Mr. Freeman is also Chair of the American Nurses Association (ANA) Advisory Board on Data Science and AI. He is pursuing his DNP at Yale University and holds a
                Master of Science in business analytics from NYU Stern School of business and a Master of Science in Nursing."""
}
event_message = [{"role": "user", "content": f"Event Description: {event_description}\n Speakers: {speaker_1}\n {speaker_2}"}]


Generate description of Event and speakers

In [4]:
messages = general_system_message + event_message
messages.append({
    "role": "user",
    "content": """ Using the event and speaker descriptions, generate a detailed description for the ideal candidates to attend this event."""
})

response = get_openai_response(messages)
ideal_candidates = response["choices"][0]["message"]["content"].strip(" \n")
messages.append({
    "role": "assistant",
    "content": ideal_candidates,
})

In [5]:
user_1_message =  """ Hi, I'm Sienna, a 25-year-old female originally from New York City, now living in California, I am currently working as a software engineer at Google.
                    I pursued my education in computer science at Stanford University, which helped cultivate my interest in AI research. My primary goal is to
                    learn more about the AI industry and its future directions, both in terms of technology and business opportunities. I am also keen on exploring
                    potential collaborations and expanding my professional network in this field. In my spare time, I enjoy staying up-to-date with the latest advancements
                    in AI and exploring innovative ways to apply this technology to various domains. """

messages.append({
    "role": "user",
    "content": f""" Ask open-ended questions to help identify the ideal inquiries that should be posed to attendees. User:  {user_1_message}""",
})

response = get_openai_response(messages)
questions = response["choices"][0]["message"]["content"].strip(" \n")
messages.append({
    "role": "assistant",
    "content": """1. What initially sparked your interest in AI, and how has your journey in this field evolved since then?

2. Can you share any specific AI projects or research you have been involved in, and what were the key takeaways from those experiences?

3. How do you envision the future of AI in healthcare, particularly in nursing practice, and what role do you think software engineers like yourself can play in this transformation?

4. Are there any challenges or ethical considerations you believe should be addressed when implementing AI in healthcare settings?

5. What are some innovative AI applications you have come across in healthcare or other industries that you think could be beneficial for nursing practice?

6. How do you stay updated on the latest advancements in AI, and are there any resources or communities you would recommend for someone looking to expand their knowledge in this area?

7. In your opinion, what are the most promising business opportunities or areas of growth within the AI industry, especially in the context of healthcare?

8. Are you currently involved in any AI-related collaborations or projects, and if so, can you share some insights into the goals and progress of these initiatives?

9. How do you balance your professional interests in AI with your personal life and hobbies, and do you find any synergies between the two?

10. As someone looking to expand their professional network in the AI field, what advice would you give for making meaningful connections and fostering collaboration at events like this symposium?""",
})

user_answers = {
    "role": "user",
    "content": """ 1. What initially sparked your interest in AI, and how has your journey in this field evolved since then?
Bayesian statistics and machine learning were the first topics that sparked my interest in AI. I was fascinated by the idea of building models that could learn from data and make predictions. Since then
I have been involved in several AI projects and research initiatives, which have helped me gain a deeper understanding of the field and its potential applications. I am particularly interested in the
business opportunities and growth areas within the AI industry, and I am looking forward to exploring these topics further in the coming years.
2. Can you share any specific AI projects or research you have been involved in, and what were the key takeaways from those experiences?
 I recently worked on an AI project called SubjectMatterAI. An AI-powered writing platform for experts, SubjectMatterAI helps experts write articles and books faster and more efficiently. I was involved the complete development 
 cycle of this project, from ideation to deployment. I was responsible for building the AI model that would generate the content for the articles and books. I also worked on the front-end development of the platform, 
 which involved building the user interface and integrating the AI model with the platform. I learned a lot from this project, especially about the importance of having a strong team and a clear vision for the project. 
 I also learned how to effectively communicate with stakeholders and manage expectations.
3. How do you envision the future of AI in healthcare, particularly in nursing practice, and what role do you think software engineers like yourself can play in this transformation?
AI has the potential to transform healthcare by improving the quality of care and reducing costs. It can also help nurses make more informed decisions and improve patient outcomes. I believe that software engineers \
    can play a key role in this transformation by building AI models that can be used to automate certain tasks and improve the efficiency of healthcare systems. I am also interested in exploring the business opportunities 
    within the AI industry, and I am looking forward to learning more about this topic at the symposium.

4. Are there any challenges or ethical considerations you believe should be addressed when implementing AI in healthcare settings?
AI models are only as good as the data they are trained on. This means that the data used to train the model can have a significant impact on the model's performance. This is especially true in healthcare, where the 
data is often incomplete and biased. I believe that these challenges can be addressed by collecting more data and ensuring that the data is representative of the population. I am also interested in exploring the ethical
considerations of AI in healthcare, and I am looking forward to learning more about this topic at the symposium.

5. What are some innovative AI applications you have come across in healthcare or other industries that you think could be beneficial for nursing practice?
I recently came across an AI-powered platform called SubjectMatterAI. This platform helps experts write articles and books faster and more efficiently. I believe that this platform could be beneficial for nursing practice 
by helping nurses write articles and books faster and more efficiently. I am also interested in exploring the business opportunities within the AI industry, and I am looking forward to learning more about this topic at the symposium.

6. How do you stay updated on the latest advancements in AI, and are there any resources or communities you would recommend for someone looking to expand their knowledge in this area?
I stay updated on the latest advancements in AI by reading articles and watching videos on YouTube. I also follow several AI experts on Twitter and LinkedIn. I would recommend that anyone looking to expand their knowledge 
in this area should follow AI experts on social media and read articles on AI-related topics. I am also interested in exploring the business opportunities within the AI industry, and I am looking forward to learning more
about this topic at the symposium.

7. In your opinion, what are the most promising business opportunities or areas of growth within the AI industry, especially in the context of healthcare?
I believe that the most promising business opportunities within the AI industry are in the areas of healthcare and education. I am also interested in exploring the business opportunities within the AI industry, and I am
looking forward to learning more about this topic at the symposium.

8. Are you currently involved in any AI-related collaborations or projects, and if so, can you share some insights into the goals and progress of these initiatives?
I recently worked on an AI project called SubjectMatterAI. An AI-powered writing platform for experts, SubjectMatterAI helps experts write articles and books faster and more efficiently. I was involved the complete development
cycle of this project, from ideation to deployment. I was responsible for building the AI model that would generate the content for the articles and books. I also worked on the front-end development of the platform, which
involved building the user interface and integrating the AI model with the platform. I learned a lot from this project, especially about the importance of having a strong team and a clear vision for the project. I also learned
how to effectively communicate with stakeholders and manage expectations.

9. How do you balance your professional interests in AI with your personal life and hobbies, and do you find any synergies between the two?
I balance my professional interests in AI with my personal life and hobbies by setting aside time for both. I also try to find synergies between my professional interests and my personal life and hobbies. For example, I am
interested in exploring the business opportunities within the AI industry, and I am looking forward to learning more about this topic at the symposium.

10. As someone looking to expand their professional network in the AI field, what advice would you give for making meaningful connections and fostering collaboration at events like this symposium?
I would recommend that anyone looking to expand their professional network in the AI field should attend events like this symposium and network with other attendees. I am also interested in exploring the business opportunities
within the AI industry, and I am looking forward to learning more about this topic at the symposium.
"""
}

In [6]:
messages.append(user_answers)
messages.append({
    "role": "user",
    "content": f""" Generate a list of engaging and quirky ice breaker questions between Sienna and Robbie.""",
})

response = get_openai_response(messages)
questions = response["choices"][0]["message"]["content"].strip(" \n")
print(questions)

1. Sienna: "If you could create an AI-powered superhero to assist nurses, what would their superpower be and why?"

2. Robbie: "As a software engineer, have you ever encountered a surprising or funny bug in an AI project that you'd like to share?"

3. Sienna: "If you could choose any fictional AI character to be your personal assistant, who would it be and why?"

4. Robbie: "What's the most unusual or unexpected way you've seen AI being used in healthcare or nursing practice?"

5. Sienna: "If you had the chance to travel back in time and give your younger self advice about pursuing a career in AI, what would you say?"

6. Robbie: "Imagine a world where AI has become an integral part of daily life. What's one mundane task you'd love for AI to take over for you?"

7. Sienna: "If you could collaborate with any historical figure on an AI project, who would it be and what would you work on together?"

8. Robbie: "What's a hobby or interest you have outside of AI and healthcare that you thin

In [7]:
messages.append({
    "role": "user",
    "content": f""" Based on the answers provided by the user, recommend three relevant and personal questions the user can ask speakers and atendees at the symposium to help facilitate meaningful connections at
                events by generating personalized and engaging questions, making it easy to break the ice and have deep conversations. \n Answers: {user_answers}""",
})

response = get_openai_response(messages)
questions = response["choices"][0]["message"]["content"].strip(" \n")
print(questions)